# co

> Covariance and Coherence Matrix Estimation

In [ ]:
#| default_exp co

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
import zarr
import moraine as mr
import math
import itertools
from moraine.utils_ import is_cuda_available
if is_cuda_available():
    import cupy as cp

In [ ]:
#| export
import math
import numpy as np
from moraine.utils_ import is_cuda_available, get_array_module
if is_cuda_available():
    import cupy as cp
from typing import Union
import moraine as mr
from moraine.utils_ import ngpjit, ngjit
from numba import prange

## Covariance and Coherence Matrix Estimator

In [ ]:
#| export
if is_cuda_available():
    _emperical_co_kernel = cp.ElementwiseKernel(
        'raw T rslc, raw bool is_shp, int32 nlines, int32 width, int32 nimages, int32 az_half_win, int32 r_half_win',
        'raw T cov, raw T coh',
        '''
        if (i >= nlines*width) return;
        int az_win = 2*az_half_win+1;
        int r_win = 2*r_half_win+1;
        int win = az_win*r_win;
        
        int ref_az = i/width;
        int ref_r = i -ref_az*width;
    
        int sec_az, sec_r;
    
        int m,j; // index of each coherence matrix
        int k,l; // index of search window
        T _cov; // covariance
        float _amp2_m; // sum of amplitude square for image i
        float _amp2_j; // sum of amplitude aquare for image j
        int rslc_inx_m, rslc_inx_j;
        int n; // number of shp
    
        for (m = 0; m < nimages; m++) {
            for (j = 0; j < nimages; j++) {
                _cov = T(0.0, 0.0);
                _amp2_m = 0.0;
                _amp2_j = 0.0;
                n = 0;
                for (k = 0; k < az_win; k++) {
                    for (l = 0; l < r_win; l++) {
                        sec_az = ref_az-az_half_win+k;
                        sec_r = ref_r-r_half_win+l;
                        if (is_shp[i*win+k*r_win+l] && sec_az >= 0 && sec_az < nlines && sec_r >= 0 && sec_r < width) {
                            rslc_inx_m = (sec_az*width+sec_r)*nimages+m;
                            rslc_inx_j = (sec_az*width+sec_r)*nimages+j;
                            _amp2_m += norm(rslc[rslc_inx_m]);
                            _amp2_j += norm(rslc[rslc_inx_j]);
                            _cov += rslc[rslc_inx_m]*conj(rslc[rslc_inx_j]);
                            n += 1;
                            //if (i == 0 && m ==3 && j == 1) {
                            //    printf("%f",_cov.real());
                            //}
                        }
                    }
                }
                cov[(i*nimages+m)*nimages+j] = _cov/(float)n;
                //if ( i == 0 && m==3 && j ==1 ) printf("%d",((i*nimages+m)*nimages+j));
                _amp2_m = sqrt(_amp2_m*_amp2_j);
                coh[(i*nimages+m)*nimages+j] = _cov/_amp2_m;
            }
        }
        ''',
        name = 'emperical_co_kernel',reduce_dims = False,no_return=True
    )

In [ ]:
#| hide
# havn't determine to use this one or not， don't use it as it launch more threads
# and havn't determine to calculate only lower part of coh and cov and copy it to the upside.
# Still worry about the index computing problem since only float64 works.
# Since float64 works slow on most GPU, compute all coh and cov. 
if is_cuda_available():
    _emperical_co_kernel1 = cp.ElementwiseKernel(
        'raw T rslc, raw bool is_shp, int32 nlines, int32 width, int32 nimages, int32 az_half_win, int32 r_half_win',
        'raw T cov, raw T coh',
        '''
        int az_win = 2*az_half_win+1;
        int r_win = 2*r_half_win+1;
        if (i >= nlines*width*az_win*r_win) return;
        int win = az_win*r_win;
        
        int dim = i;
        int j = dim%nimages; // (ref_az, ref_r, m, j)
        dim = dim/nimages;
        int m = dim%nimages;
        dim = dim/nimages;
        int ref_r = dim%width;
        dim = dim/width;
        int ref_az = dim%nlines;
        
        T _cov = T(0.0, 0.0); // covariance
        float _amp2_m = 0; // sum of amplitude square for image i
        float _amp2_j = 0; // sum of amplitude aquare for image j
        int sec_az, sec_r;
    
        int n = 0;
        int k,l; // index of search window
        int rslc_inx_m, rslc_inx_j;
    
        for (k = 0; k < az_win; k++) {
            for (l = 0; l < r_win; l++) {
                sec_az = ref_az-az_half_win+k;
                sec_r = ref_r-r_half_win+l;
                if (is_shp[ref_az*width*win+ref_r*win+k*r_win+l] && sec_az >= 0 && sec_az < nlines && sec_r >= 0 && sec_r < width) {
                    rslc_inx_m = (sec_az*width+sec_r)*nimages+m;
                    rslc_inx_j = (sec_az*width+sec_r)*nimages+j;
                    _amp2_m += norm(rslc[rslc_inx_m]);
                    _amp2_j += norm(rslc[rslc_inx_j]);
                    _cov += rslc[rslc_inx_m]*conj(rslc[rslc_inx_j]);
                    n += 1;
                }
            }
        }
        int co_idx = ((ref_az*width+ref_r)*nimages+m)*nimages+j;
        cov[co_idx] = _cov/(float)n;
        _amp2_m = sqrt(_amp2_m*_amp2_j);
        coh[co_idx] = _cov/_amp2_m;
        ''',
        name = 'emperical_co_kernel',reduce_dims = False,no_return=True
    )
    def emperical_co1(rslc:cp.ndarray, # rslc stack, dtype: `cupy.complexfloating`
                     is_shp:cp.ndarray, # shp bool, dtype: `cupy.bool`
                     block_size:int=128, # the CUDA block size, it only affects the calculation speed
                    )-> tuple[cp.ndarray,cp.ndarray]: # the covariance and coherence matrix `cov` and `coh`
        '''
        Maximum likelihood covariance estimator.
        '''
        nlines, width, nimages = rslc.shape
        az_win, r_win = is_shp.shape[-2:]
        az_half_win = (az_win-1)//2
        r_half_win = (r_win-1)//2
    
        cov = cp.empty((nlines,width,nimages,nimages),dtype=rslc.dtype)
        coh = cp.empty((nlines,width,nimages,nimages),dtype=rslc.dtype)
    
        _emperical_co_kernel(rslc, is_shp, cp.int32(nlines),cp.int32(width),cp.int32(nimages),
                        cp.int32(az_half_win),cp.int32(r_half_win),cov,coh,size = nlines*width*az_win*r_win,block_size=block_size)
        return cov,coh

In [ ]:
#| export
def emperical_co(rslc:np.ndarray, # rslc stack, dtype: `cupy.complexfloating`
                 is_shp:np.ndarray, # shp bool, dtype: `cupy.bool`
                 block_size:int=128, # the CUDA block size, it only affects the calculation speed
                )-> tuple[np.ndarray,np.ndarray]: # the covariance and coherence matrix `cov` and `coh`
    '''
    Maximum likelihood covariance estimator.
    '''
    xp = get_array_module(rslc)
    if xp is np:
        raise NotImplementedError("Currently only cuda version available.")
    nlines, width, nimages = rslc.shape
    az_win, r_win = is_shp.shape[-2:]
    az_half_win = (az_win-1)//2
    r_half_win = (r_win-1)//2

    cov = cp.empty((nlines,width,nimages,nimages),dtype=rslc.dtype)
    coh = cp.empty((nlines,width,nimages,nimages),dtype=rslc.dtype)

    _emperical_co_kernel(rslc, is_shp, cp.int32(nlines),cp.int32(width),cp.int32(nimages),
                    cp.int32(az_half_win),cp.int32(r_half_win),cov,coh,size = nlines*width,block_size=block_size)
    return cov,coh

::: {.callout-warning}
This function is deprecated as estimate coherence for all pixels are not necessary. Please use `emperical_co_pc` instead.
:::

The `cov` and `coh` is defined as:

$$
cov = E(z_1z_2^*) \quad coh=\frac{E(z_1z_2^*)}{\sqrt{E(|z_1|^2)E(|z_2|^2)}}
$$

where $z_1$ and $z_2$ are the reference and secondary SLC images, respectively. `cov` and `coh` are estimated as:

$$
cov = \frac{\sum_{i=1}^{L}z_1^{i}z_2^{i*}}{L} \quad coh = \frac{\sum_{i=1}^{L}z_1^{i}z_2^{i*}}{\sqrt(\sum_{i=1}^{L}|z_1^{i}|^2)(\sum_{i=1}^{L}|z_2^{i}|^2)}
$$

using all selected SHPs. Their shapes are [nlines,width,nimages,nimages]. Note dim 2 and dim 3 are reference images and secondary images, respectively.

The `rslc` is a three dimentional cupy `ndarray`. The `dtype` should be `cupy.complex64`. From outerest to innerest, the three dimentions are azimuth, range and image.
`is_shp` is a four dimentional cupy `ndarray`. It describes if pixels in the search window are SHP to the central pixel.
From outerest ot innerest, they are azimuth, range, secondary pixel relative azimuth, secondary pixel relative range.

Here is an example:

In [ ]:
# synthetic data
rslc = np.random.rand(5,10,17).astype(np.float32) + 1j*np.random.rand(5,10,17).astype(np.float32)
half_az_win = 1;
half_r_win = 2;
is_shp = np.random.choice(a=[False, True], size=(5,10,2*half_az_win+1,2*half_r_win+1), p=[0.3, 0.7])
for i, j, k, l in itertools.product(range(is_shp.shape[0]),range(is_shp.shape[1]),range(is_shp.shape[2]),range(is_shp.shape[3])):
    if (k == half_az_win) and (l == half_r_win):
        is_shp[i,j,k,l] = True
    if (i+k-half_az_win<0) or (i+k-half_az_win>=rslc.shape[0]) or (j+l-half_r_win<0) or (j+l-half_r_win>=rslc.shape[1]):
        is_shp[i,j,k,l] = False

In [ ]:
print(rslc.shape, is_shp.shape, is_shp[2,3])

(5, 10, 17) (5, 10, 3, 5) [[ True  True False False  True]
 [ True  True  True  True False]
 [ True False False  True  True]]


`rslc` is a stack of 17 rslc images. Each of the image has 5 pixel in azimuth dimention and 10 pixels in range dimention.
It shows for pixel (2,3), the (3*5) window around it has 2 SHPs to it (the central one is itself).

In [ ]:
if is_cuda_available():
    rslc_cp = cp.asarray(rslc)
    is_shp_cp = cp.asarray(is_shp)
    cov_cp,coh_cp = emperical_co(rslc_cp,is_shp_cp)
    print(cov_cp.shape, coh_cp.shape)

(5, 10, 17, 17) (5, 10, 17, 17)


Both `cov` and `coh` are complex data. The shape shows each covarience or coherence matrix is 17 by 17 since there are 17 images.
And `cov` and `coh` are matrix for all 5*10 pixels.

In [ ]:
#| hide
# test
if is_cuda_available():
    half_az_win = is_shp.shape[2]//2;
    half_r_win = is_shp.shape[3]//2;
    for i, j, k, l in itertools.product(range(rslc.shape[0]),range(rslc.shape[1]),range(rslc.shape[2]),range(rslc.shape[2])):
        _cov = 0.0+0.0j
        _amp2_k = 0.0
        _amp2_l = 0.0
        # shp_az, shp_r
        n_shp = 0
        for m, n in itertools.product(range(is_shp.shape[2]),range(is_shp.shape[3])):
            if is_shp[i,j,m,n]:
                _cov += rslc[i+m-half_az_win,j+n-half_r_win,k]*rslc[i+m-half_az_win,j+n-half_r_win,l].conj()
                _amp2_k += abs(rslc[i+m-half_az_win,j+n-half_r_win,k])**2
                _amp2_l += abs(rslc[i+m-half_az_win,j+n-half_r_win,l])**2
                n_shp+=1
        assert abs(_cov/n_shp-cov_cp[i,j,k,l])<1.0e-6
        assert abs(_cov/math.sqrt(_amp2_k*_amp2_l) - coh_cp[i,j,k,l]) < 1.0e-6

In [ ]:
#| export
if is_cuda_available():
    _emperical_co_pc_kernel = cp.ElementwiseKernel(
        'raw T rslc, raw I az_idx, raw I r_idx, raw bool pc_is_shp, raw I image_pair_ref, raw I image_pair_sec, int32 nlines, int32 width, int32 nimages, int32 az_half_win, int32 r_half_win, int32 n_pc, int32 n_image_pair',
        'raw T coh',
        '''
        if (i >= n_pc) return;
        int az_win = 2*az_half_win+1;
        int r_win = 2*r_half_win+1;
        int win = az_win*r_win;
        
        int ref_az = az_idx[i];
        int ref_r = r_idx[i];
    
        int sec_az, sec_r;
    
        int m,j; // index of each coherence matrix
        int pair_i; // index of image pairs
        int k,l; // index of search window
        T _co_nume; // covariance/coherence numerator
        T _coh; // coherence
        float _amp2_m; // sum of amplitude square for image i
        float _amp2_j; // sum of amplitude aquare for image j
        int rslc_inx_m, rslc_inx_j;

        for (pair_i=0; pair_i < n_image_pair; pair_i++){
            m = image_pair_ref[pair_i];
            j = image_pair_sec[pair_i];
            
            _co_nume = T(0.0, 0.0);
            _amp2_m = 0.0;
            _amp2_j = 0.0;
            for (k = 0; k < az_win; k++) {
                for (l = 0; l < r_win; l++) {
                    sec_az = ref_az-az_half_win+k;
                    sec_r = ref_r-r_half_win+l;
                    if (pc_is_shp[i*win+k*r_win+l] && sec_az >= 0 && sec_az < nlines && sec_r >= 0 && sec_r < width) {
                        rslc_inx_m = (sec_az*width+sec_r)*nimages+m;
                        rslc_inx_j = (sec_az*width+sec_r)*nimages+j;
                        _amp2_m += norm(rslc[rslc_inx_m]);
                        _amp2_j += norm(rslc[rslc_inx_j]);
                        _co_nume += rslc[rslc_inx_m]*conj(rslc[rslc_inx_j]);
                    }
                }
            }
            _amp2_m = sqrt(_amp2_m*_amp2_j);
            _coh = _co_nume/_amp2_m;
            coh[i*n_image_pair+pair_i] = _coh;
        }
        ''',
        name = 'emperical_co_pc_kernel',reduce_dims = False,no_return=True
    )

In [ ]:
#| export
@ngpjit
def _emperical_co_pc_numba(
    rslc,
    az_idx,
    r_idx,
    pc_is_shp,
    image_pairs,
):
    nlines, width, nimages = rslc.shape
    n_pc = az_idx.shape[0]
    az_win, r_win = pc_is_shp.shape[1:]
    az_half_win, r_half_win = az_win//2, r_win//2
    npairs = image_pairs.shape[0]

    coh = np.empty((n_pc, npairs),dtype=rslc.dtype)

    for i in prange(n_pc):
        for pair_i in range(npairs):
            m, j = image_pairs[pair_i]
            _co_nume = 0.0 + 0.0j
            _amp2_m = 0.0
            _amp2_j = 0.0
            for k in range(az_win):
                for l in range(r_win):
                    az_idx_ = az_idx[i] - az_half_win + k
                    r_idx_ = r_idx[i] - r_half_win + l
                    if (az_idx_ >= 0) and (az_idx_ < nlines) and (r_idx_ >= 0) and (r_idx_ < width) and pc_is_shp[i,k,l]:
                        rslc_m = rslc[az_idx_, r_idx_, m]
                        rslc_j = rslc[az_idx_, r_idx_, j]
                        _amp2_m += rslc_m.real**2 + rslc_m.imag**2
                        _amp2_j += rslc_j.real**2 + rslc_j.imag**2
                        _co_nume += rslc_m*np.conj(rslc_j)
            _coh = _co_nume/math.sqrt(_amp2_m*_amp2_j)
            coh[i,pair_i] = _coh
    return coh

In [ ]:
#| export
def emperical_co_pc(rslc:np.ndarray, # rslc stack, dtype: `np.complex64`
                    idx:np.ndarray, # index of point target (azimuth_index, range_index), dtype: `np.int32`, shape: (n_pc, 2)
                    pc_is_shp:np.ndarray, # shp bool, dtype: `bool`
                    block_size:int=128, # the CUDA block size, it only affects the calculation speed
                    image_pairs:np.ndarray=None, # only coherence of those image pairs will estimated, dtype: `np.int32`, shape: (n_image_pair, 2)
                   )-> np.ndarray: # `coh`, dtype:`np.complex64`, shape(n_pc, n_image_pair)
    '''
    Maximum likelihood covariance estimator for sparse data.
    '''
    xp = get_array_module(rslc)
    nlines, width, nimages = rslc.shape
    az_win, r_win = pc_is_shp.shape[-2:]
    az_half_win = (az_win-1)//2
    r_half_win = (r_win-1)//2
    idx = idx.astype(np.int32)
    az_idx = idx[:,0]; r_idx = idx[:,1]
    n_pc = az_idx.shape[0]

    if image_pairs is None:
        image_pairs = mr.TempNet.from_bandwidth(nimages).image_pairs
    image_pairs = image_pairs.astype(np.int32)

    if xp is np:
        return _emperical_co_pc_numba(rslc,az_idx,r_idx,pc_is_shp,image_pairs)

    else:
        image_pairs = cp.asarray(image_pairs)
        coh = cp.empty((n_pc,image_pairs.shape[0]),dtype=rslc.dtype)
        _emperical_co_pc_kernel(
            rslc, az_idx, r_idx, pc_is_shp, image_pairs[:,0], image_pairs[:,1],
            cp.int32(nlines),cp.int32(width),cp.int32(nimages),
            cp.int32(az_half_win),cp.int32(r_half_win),
            cp.int32(n_pc), cp.int32(image_pairs.shape[0]),
            coh,
            size = n_pc,block_size=block_size
        )
        return coh

`emperical_co_sp` is the `emperical_co` on sparse data, e.g., DSs. `rslc` is same as `emperical_co`. `sp_idx` is the index, i.e., a tuple of (azimuth_idx, range_idx). Each index is 1D array. `pc_is_shp` is similar to `is_shp` in `emperical_co` but it only contains information about the point cloud data. It is a 3D array with shape [number_of_point,az_win,r_win].

Compared with `emperical_co`, `emperical_co_pc` only estimate coherence at specific position so the memory usage is much small.

`emperical_co` only return the coherence of specified image pairs to save data volume (compressed coherence).
By default, only the upper triangle (with offset=1) of the coherence matrix is returned.

Example:

In [ ]:
# synthetic data
rslc = np.random.rand(5,10,17).astype(np.float32) + 1j*np.random.rand(5,10,17).astype(np.float32)
half_az_win = 1;
half_r_win = 2;

is_shp = np.random.choice(a=[False, True], size=(5,10,2*half_az_win+1,2*half_r_win+1), p=[0.9, 0.1])
# ensure the format of is_shp is correct:
for i, j, k, l in itertools.product(range(is_shp.shape[0]),range(is_shp.shape[1]),range(is_shp.shape[2]),range(is_shp.shape[3])):
    if (k == half_az_win) and (l == half_r_win):
        is_shp[i,j,k,l] = True
    if (i+k-half_az_win<0) or (i+k-half_az_win>=rslc.shape[0]) or (j+l-half_r_win<0) or (j+l-half_r_win>=rslc.shape[1]):
        is_shp[i,j,k,l] = False

shp_num = np.count_nonzero(is_shp,axis=(-2,-1))
is_ds_can = shp_num >= 3
ds_can_is_shp = is_shp[is_ds_can]
ds_can_idx = np.stack(np.where(is_ds_can),axis=-1)

print(rslc.shape,ds_can_idx.shape,ds_can_is_shp.shape)

(5, 10, 17) (19, 2) (19, 3, 5)


`rslc` is a stack of 17 rslc images. Each of the image has 5 pixel in azimuth dimention and 10 pixels in range dimention.
`ds_can_idx` shows the index of the DS candidates and `ds_can_is_shp` shows the corrosponding SHPs.

In [ ]:
ds_can_coh = emperical_co_pc(rslc,ds_can_idx,ds_can_is_shp)

In [ ]:
ds_can_coh.shape

(19, 136)

The shape of the coherence matrix is (17,17) while only the upper triangle of it is returned (136 elements).

Or, we can specify the image pairs:

In [ ]:
tempnet = mr.TempNet.from_bandwidth(rslc.shape[0],bandwidth=3)
print(tempnet.image_pairs.shape) # 9 rslc image pairs
ds_can_coh = emperical_co_pc(rslc,ds_can_idx,ds_can_is_shp,image_pairs=tempnet.image_pairs)
print(ds_can_coh.shape)

(9, 2)
(19, 9)


Input `cupy.ndarray` is also supported:

In [ ]:
if is_cuda_available():
    rslc_cp = cp.asarray(rslc)
    ds_can_idx_cp = cp.asarray(ds_can_idx)
    ds_can_is_shp_cp = cp.asarray(ds_can_is_shp)
    ds_can_coh_cp = emperical_co_pc(rslc_cp,ds_can_idx_cp,ds_can_is_shp_cp)

In [ ]:
#| hide
#| test if cpu and gpu version generate similar result
if is_cuda_available():
    ds_can_coh = emperical_co_pc(rslc,ds_can_idx,ds_can_is_shp)
    rslc_cp = cp.asarray(rslc)
    ds_can_idx_cp = cp.asarray(ds_can_idx)
    ds_can_is_shp_cp = cp.asarray(ds_can_is_shp)
    ds_can_coh_cp = emperical_co_pc(rslc_cp,ds_can_idx_cp,ds_can_is_shp_cp)
    np.testing.assert_array_almost_equal(ds_can_coh, cp.asnumpy(ds_can_coh_cp))

In [ ]:
#| hide
if is_cuda_available():
    #| test if return_cov change the returned coh for gpu version
    rslc_cp = cp.asarray(rslc)
    is_shp_cp = cp.asarray(is_shp)
    ds_can_idx_cp = cp.asarray(ds_can_idx)
    ds_can_is_shp_cp = cp.asarray(ds_can_is_shp)
    ds_can_coh_cp = emperical_co_pc(rslc_cp,ds_can_idx_cp,ds_can_is_shp_cp)

    #| test if emperical_co and emperical_co_pc generate same result
    cov_cp,coh_cp = emperical_co(rslc_cp,is_shp_cp)
    pairs = cp.triu_indices(rslc_cp.shape[2],1)
    cp.testing.assert_array_almost_equal(coh_cp[is_ds_can][:,pairs[0],pairs[1]],ds_can_coh_cp)

In [ ]:
#| export
#| hide
@ngjit
def uncompress_single_coh_numba(coh, nimages, image_pairs):
    uncompressed_coh = np.zeros((nimages,nimages),dtype=coh.dtype)
    ref_images, sec_images = image_pairs[:,0], image_pairs[:,1]
    for i in range(ref_images.shape[0]):
        uncompressed_coh[ref_images[i], sec_images[i]] = coh[i]
        uncompressed_coh[sec_images[i], ref_images[i]] = np.conj(coh[i])
    for i in range(nimages):
        uncompressed_coh[i,i] = 1
    return uncompressed_coh

In [ ]:
#| export
def uncompress_coh(
    coh:np.ndarray, # compressed coherence stack, dtype: `np.complex64`, shape(...,n_image_pair)
    image_pairs:np.ndarray=None, # image pairs, dtype: `np.int32`, shape: (n_image_pair, 2), all pairs by default
)-> np.ndarray: # uncompressed, dtype:`np.complex64`, shape(..., n_image_pair)
    '''
    uncompress coh matrix to a hermitian matrix
    '''
    xp = get_array_module(coh)
    if image_pairs is None:
        nimages = mr.nimage_from_npair(coh.shape[-1])
        image_pairs = mr.TempNet.from_bandwidth(nimages).image_pairs
    else:
        nimages = image_pairs[-1,-1]+1
    uncompressed_coh = xp.zeros((*coh.shape[:-1],nimages,nimages),dtype=coh.dtype)
    ref_images, sec_images = image_pairs[:,0], image_pairs[:,1]
    uncompressed_coh[...,ref_images,sec_images] = coh
    uncompressed_coh[...,sec_images,ref_images] = coh.conj()
    uncompressed_coh[...,np.arange(nimages),np.arange(nimages)] = 1
    return uncompressed_coh

Example:

In [ ]:
nimages = 5
tempnet = mr.TempNet.from_bandwidth(nimages,bandwidth=2)
print(tempnet.image_pairs.shape)
coh = np.random.rand(tempnet.image_pairs.shape[0]).astype(np.float32) + 1j*np.random.rand(tempnet.image_pairs.shape[0]).astype(np.float32)
uncompressed_coh = uncompress_coh(coh,tempnet.image_pairs)
uncompressed_coh.real

(7, 2)


array([[1.        , 0.75777644, 0.05921678, 0.        , 0.        ],
       [0.75777644, 1.        , 0.11659877, 0.44703874, 0.        ],
       [0.05921678, 0.11659877, 1.        , 0.8556764 , 0.3602692 ],
       [0.        , 0.44703874, 0.8556764 , 1.        , 0.84126586],
       [0.        , 0.        , 0.3602692 , 0.84126586, 1.        ]],
      dtype=float32)

## Covariance and Coherence Matrix Regularizer

In [ ]:
#| export
def isPD(co:np.ndarray, # absolute value of complex coherence/covariance stack
         )-> np.ndarray: # bool array indicating wheather coherence/covariance is positive define
    xp = get_array_module(co)
    L = xp.linalg.cholesky(co)
    is_PD = xp.isfinite(L).all(axis=(-2,-1))
    return is_PD

::: {.callout-warning}
This function is deprecated.
:::

This function tells if the matrix is positive defined or not. 

In [ ]:
#| code-fold: true
#| code-summary: "Code for generating data for doc"
rslc = zarr.open('../../data/rslc.zarr/','r')[600:650,600:650]
if is_cuda_available():
    rslc = cp.asarray(rslc)
    
    # SHP selection
    az_half_win = 5; r_half_win = 5
    az_win = 2*az_half_win+1; r_win = 2*r_half_win+1
    
    rmli = cp.abs(rslc)**2
    sorted_rmli = cp.sort(rmli,axis=-1)
    p = mr.ks_test(sorted_rmli,az_half_win=az_half_win,r_half_win=r_half_win)
    is_shp = p < 0.05
    
    # Select DS candidate
    shp_num = cp.count_nonzero(is_shp,axis=(-2,-1))
    is_ds_can = shp_num >= 50
    ds_can_is_shp = is_shp[is_ds_can]
    ds_can_idx = cp.stack(cp.where(is_ds_can),axis=-1)
    
    ds_can_coh = uncompress_coh(emperical_co_pc(rslc,ds_can_idx,ds_can_is_shp))

In [ ]:
if is_cuda_available():
    print(ds_can_coh.shape)

(149, 17, 17)


In [ ]:
if is_cuda_available():
    isPD_ds_can = isPD(ds_can_coh)
    print(isPD_ds_can)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True]


All coherence matrix are positive defined.

In [ ]:
#| export
'''
    The method is presented in [1]. John D'Errico implented it in MATLAB [2] under BSD
    Licence and [3] implented it with Python/Numpy based on [2] also under BSD Licence.
    This is a cupy implentation with stack of matrix supported.

    [1] N.J. Higham, "Computing a nearest symmetric positive semidefinite
    matrix" (1988): https://doi.org/10.1016/0024-3795(88)90223-6
    
    [2] https://www.mathworks.com/matlabcentral/fileexchange/42885-nearestspd
    
    [3] https://gist.github.com/fasiha/fdb5cec2054e6f1c6ae35476045a0bbd
'''
def nearestPD(co:np.ndarray, # stack of matrix with shape [...,N,N]
             )-> np.ndarray: # nearest positive definite matrix of input, shape [...,N,N]
    """Find the nearest positive-definite matrix to input matrix."""
    xp = get_array_module(co)
    B = (co + xp.swapaxes(co,-1,-2))/2
    s, V = xp.linalg.svd(co)[1:]
    I = xp.eye(co.shape[-1],dtype=co.dtype)
    S = s[...,None]*I
    del s

    H = xp.matmul(xp.swapaxes(V,-1,-2), xp.matmul(S, V))
    del S, V
    A2 = (B + H) / 2
    del B, H
    A3 = (A2 + xp.swapaxes(A2,-1,-2))/2
    del A2

    if wherePD(A3).all():
        return A3
    
    co_norm = xp.linalg.norm(co,axis=(-2,-1))
    spacing = xp.nextafter(co_norm,co_norm+1.0)-co_norm
    
    k = 0
    while True:
        isPD = wherePD(A3)
        isPD_all = isPD.all()
        if isPD_all or k>=100:
            break
        k+=1
        mineig = xp.amin(xp.linalg.eigvalsh(A3),axis=-1)
        assert xp.isfinite(mineig).all()
        A3 += (~isPD[...,None,None] * I) * (-mineig * k**2 + spacing)[...,None,None]
    #print(k)
    return A3

`nearest` means the Frobenius norm of the difference is minimized.

In [ ]:
#| export
def regularize_spectral(coh:np.ndarray, # stack of matrix with shape [...,N,N]
                        beta:Union[float, np.ndarray], # the regularization parameter, a float number or cupy ndarray with shape [...]
                        )-> np.ndarray: # regularized matrix, shape [...,N,N]
    '''
    Spectral regularizer for coherence matrix.
    '''
    xp = get_array_module(coh)
    I = xp.eye(coh.shape[-1],dtype=coh.dtype)
    beta = xp.asarray(beta)[...,None,None]

    regularized_coh = (1-beta)*coh + beta* I
    return regularized_coh

::: {.callout-warning}
This function is deprecated.
:::

`regularize_spectral` can regularize the absolute value of coherence matrix for better phase linking.
It is first presented in [@zwiebackCheapValidRegularizers2022a].

Examples:

In [ ]:
#| code-fold: true
#| code-summary: "Code for generating data for doc"
rslc = zarr.open('../../data/rslc.zarr/','r')[600:605,600:610]
if is_cuda_available():
    rslc = cp.asarray(rslc)
    
    # SHP selection
    az_half_win = 1; r_half_win = 2
    az_win = 2*az_half_win+1; r_win = 2*r_half_win+1
    
    rmli = cp.abs(rslc)**2
    sorted_rmli = cp.sort(rmli,axis=-1)
    p = mr.ks_test(sorted_rmli,az_half_win=az_half_win,r_half_win=r_half_win)
    is_shp = p < 0.05
    
    cov,coh = emperical_co(rslc,is_shp)

In [ ]:
if is_cuda_available():
    print(coh.shape)

(5, 10, 17, 17)


In [ ]:
if is_cuda_available():
    regularized_coh1 = regularize_spectral(coh,0.1)

More general, `bata` can be a `cp.ndarray`:

In [ ]:
if is_cuda_available():
    beta = cp.ones(coh.shape[:-2])/10
    regularized_coh2 = regularize_spectral(coh,beta)

In [ ]:
#| hide
if is_cuda_available():
    cp.testing.assert_array_almost_equal(regularized_coh1,regularized_coh2)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()